# Import 

In [1]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sqrt
from qutip import *
import time

from numpy import min, max
from scipy import linspace
import scipy.signal

Using matplotlib backend: TkAgg


# Functions

In [2]:
class ExpParams(object):
    """ A placehoder for parameters """
    pass

def plot_eps_vs_time(tlist, eps_list, params):
    """
    Plot energy vs time
    """
    
    H0 = - params.delta/2.0 * sigmax()
    eigen = [(H0 - eps/2.0 * sigmaz()).eigenstates() for eps in eps_list]

    upper_band = [eig[0][0] for eig in eigen]
    lower_band = [eig[0][1] for eig in eigen]

    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(tlist, upper_band, tlist, lower_band)
    ax.set_xlabel('Time (us)', fontsize=20)
    ax.set_ylabel('eps (MHz)', fontsize=20)
    ax.tick_params(axis='x', labelsize=18)
    ax.tick_params(axis='y', labelsize=18)
    ax.set_title('Landau-Zener transition')
    ax.grid()

# def qubit_evolve(psi0, eps_list, tlist, H_args, n_th=0):
#     """
#     eps_t : Function that represents eps(t) in -eps(t)/2*sz, 
#             i.e., defines how energy difference changes
#     eps_list : numpy array of eps list with float type
#     Return: expectation values
#     """
#     # Hamiltonian
#     sz = sigmaz()
#     sm = destroy(2)

#     H0 = - H_args['delta']/2.0 * sigmax() # - eps0/2.0 * sigmaz()     

#     # collapse operators
#     c_op_list = []
# #     n_th = 0.0 # zero temperature

#     # relaxation
#     rate1 = H_args['gamma1'] * (1 + n_th)
#     # excitation
#     rate2 = H_args['gamma1'] * n_th
#     # dephasing 
#     rate3 = H_args['gamma2']
    
#     c_op_list = [sqrt(rate1) * sm, sqrt(rate2) * sm.dag(), sqrt(rate3) * sz]
# #     eps_list = np.array([eps_t(t, H_args) for t in tlist], dtype=float) 
#     H = [H0, [-1/2.0 * sz, eps_list]]
#     output = mesolve(H, psi0, tlist, c_op_list, [sm.dag() * sm], args=H_args)  

#     return output.expect[0]

def qubit_evolve(tlist, eps_list, params):
    """
    eps_t : Function that represents eps(t) in -eps(t)/2*sz, 
            i.e., defines how energy difference changes
    eps_list : numpy array of eps list with float type
    Return: expectation values
    """
    # Hamiltonian
    sz = sigmaz()
    sm = destroy(2)

    H0 = - params.delta / 2.0 * sigmax() # - eps0/2.0 * sigmaz()     

    # collapse operators
    c_op_list = []
#     n_th = 0.0 # zero temperature

    # relaxation
    rate1 = params.gamma1 * (1 + params.n_th)
    # excitation
    rate2 = params.gamma1 * params.n_th
    # dephasing 
    rate3 = params.gamma2
    
    c_op_list = [sqrt(rate1) * sm, sqrt(rate2) * sm.dag(), sqrt(rate3) * sz]
#     eps_list = np.array([eps_t(t, H_args) for t in tlist], dtype=float) 
    H = [H0, [-1/2.0 * sz, eps_list]]
    output = mesolve(H, params.psi0, tlist, c_op_list, [sm.dag() * sm])  

    return output.expect[0]

# def plot_probality_time(eps_t):
#     eps_list = [eps_t(t, H_args) for t in tlist]
#     fig, (ax,ax1) = plt.subplots(2,1,figsize=(12,10))
#     ax.plot(tlist, np.real(p_ex), 'b', tlist, np.real(1-p_ex), 'r')
#     ax.set_xlabel('Time (us)', fontsize=20)
#     ax.set_ylabel('Occupation probability', fontsize=20)
#     ax.tick_params(axis='x', labelsize=18)
#     ax.tick_params(axis='y', labelsize=18)
#     ax.set_title(f'Pulse out and idle, \n f1={f1:.3f}, df01/dt={df01df*dfdt:.1f} (MHz/us), \n df/dt={dfdt*1e3:.3f} (mPhi0/us), delta={delta:.3f} MHz, Gamma1={gamma1:.3f}, Gamma2={gamma2:.3f}', fontsize=16)
#     ax.legend(("R state", "L state"), loc=0, fontsize=20);
#     ax.grid()

#     ax1.plot(tlist, eps_list)
#     ax1.set_xlabel('Time (us)', fontsize=20)
#     ax1.set_ylabel('eps (MHz)', fontsize=20)
#     ax1.tick_params(axis='x', labelsize=18)
#     ax1.tick_params(axis='y', labelsize=18)
#     ax1.legend(("R state", "L state"), loc=0);
#     ax1.grid() 

# Pulse out

In [3]:
# Linear sweep from f1 to -f1

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
params.dfdt = 0.01/10 # in Phi0/us
params.f1 = -0.002
params.dt = 0.002    # us
params.gamma1 = 0  # relaxation rate
params.gamma2 = 0  # dephasing  rate
params.n_th = 0 
params.psi0 = basis(2,0)      # initial state


tlist = np.linspace(0,2*np.absolute(params.f1)/params.dfdt
                    , np.round(2*np.absolute(params.f1)/params.dfdt/params.dt))
eps_t = lambda t, params : (params.dfdt * t + params.f1) * params.df01df
eps_list = np.array([eps_t(t, params) for t in tlist], dtype=float)

start_time = time.time()
p_ex = qubit_evolve(tlist, eps_list, params)
print('time elapsed = ' + str(time.time() - start_time))

# plot
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(tlist, np.real(p_ex), 'b', tlist, np.real(1-p_ex), 'r')
ax.plot(tlist, 1 - np.exp(-np.pi * params.delta **2 / (2 * params.df01df*params.dfdt)) * np.ones(shape(tlist)), 'k')
ax.set_xlabel('Time (us)', fontsize=20)
ax.set_ylabel('Occupation probability', fontsize=20)
ax.set_title('Landau-Zener transition', fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.legend(("R state", "L state", "Landau-Zener formula"), loc=0, fontsize=20);
ax.grid()

plot_eps_vs_time(tlist, eps_list, params)

time elapsed = 0.1496877670288086


In [4]:
from scipy.constants import h, hbar 
t_list = np.linspace(10,1,101)*1e-9
P_LZ = np.exp(-2 * np.pi * (h*200e6)**2 / (hbar * (h*500e6/t_list)))
plt.plot(t_list/1e-9, P_LZ, label='s')
plt.xlabel('time (ns)')
plt.ylabel('P_LZ')
plt.legend()

# Pulse out and idle

In [9]:
# pulse and idle
def eps_t(t, params):
    if t <= params.t1:
        f = t * params.dfdt + params.f1
    else:
        f = params.f2
        
    eps = params.df01df * f
    
    return eps

# parameters
params =ExpParams()
params.delta = 100   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
params.dfdt = 0.01/10 # in Phi0/us
params.f1 = -0.003
params.f2 = -0.000
params.dt = 0.001    # us
params.gamma1 = 0.1  # relaxation rate
params.gamma2 = 0.1  # dephasing  rate
params.n_th = 0 
params.psi0 = basis(2,0)      # initial state
params.idle = 0.2

# compute
params.t1 = (params.f2-params.f1)/ params.dfdt
t1 = params.t1
dt = params.dt
tlist1 = np.linspace(0, t1, np.round(t1/dt)+1)
tlist2 = np.linspace(0, params.idle, np.round(params.idle/dt)+1)
tlist = np.concatenate((tlist1, tlist2[1:]+t1))
eps_list = np.array([eps_t(t, params) for t in tlist], dtype=float)

start_time = time.time()
p_ex = qubit_evolve(tlist, eps_list, params)
print('time elapsed = ' + str(time.time() - start_time))

# plot
fig, (ax,ax1) = plt.subplots(2,1,figsize=(12,10))
ax.plot(tlist, np.real(p_ex), 'b', tlist, np.real(1-p_ex), 'r')
ax.set_xlabel('Time (us)', fontsize=20)
ax.set_ylabel('Occupation probability', fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Pulse out and idle, \n f1={params.f1:.3f}, df01/dt={params.df01df*params.dfdt:.1f} (MHz/us), \n df/dt={params.dfdt*1e3:.3f} (mPhi0/us), delta={params.delta:.3f} MHz, Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f}', fontsize=16)
ax.legend(("R state", "L state"), loc=0, fontsize=20);
ax.grid()

ax1.plot(tlist, eps_list)
ax1.set_xlabel('Time (us)', fontsize=20)
ax1.set_ylabel('eps (MHz)', fontsize=20)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.legend(("R state", "L state"), loc=0);
ax1.grid()

plot_eps_vs_time(tlist, eps_list, params)
params.df01df * params.dfdt

time elapsed = 0.1855931282043457


333.33333333333337

In [ ]:
df01df*dfdt

In [39]:
# f2 vs idle , 2D

# pulse and idel
def eps_t(t, args):
    if t <= args['t1']:
        f = t * args['dfdt'] + args['f1']        
    else:
        f = args['f2']
        
    eps = args['df01df'] * f
    
    return eps

# parameters
delta = 10   #  MHz
df01df= 5000/0.02 # define energy vs flux slope MHz/Phi0
dfdt = 0.01/10  # in Phi0/us
f1 = -0.001
dt = 0.005    # us
gamma1 = 0    # relaxation rate
gamma2 = 0     # dephasing  rate

# compute
psi0 = basis(2,0)      # initial state
f2_list = np.linspace(-0.0002, 0.0002, 11) 
data = np.ones((f2_list.size, np.int(np.round(idle/dt)+1)))
for i, f2 in enumerate(f2_list):
    t1 = (f2-f1)/ dfdt
    tlist1 = np.linspace(0, t1, np.round(t1/dt)+1)
    tlist2 = np.linspace(0, idle, np.round(idle/dt)+1)
    tlist = np.concatenate((tlist1, tlist2[1:]+t1))
    exp_args ={'dfdt':dfdt, 'df01df':df01df, 't1':t1, 'f1':f1,'f2':f2, 't1':t1}
    eps_list = np.array([eps_t(t, exp_args) for t in tlist], dtype=float)
    H_args = {'delta':delta, 'gamma1':gamma1, 'gamma2':gamma2}
    p_ex = qubit_evolve(psi0, eps_list, tlist, H_args)
    data[i,:] = p_ex[-np.int(np.round(idle/dt))-2:-1]

# plot
idle_mat, f2_mat = np.meshgrid(tlist2, f2_list)
fig, ax = plt.subplots(figsize=(12,8))
ax.pcolor(idle_mat, f2_mat*1e3, data)
ax.set_xlabel('Idle time (us)',fontsize=20)
ax.set_ylabel(r'$\Delta$Flux (m$\Phi_0$)',fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Pulse out and idle, delta={delta} (MHz), f1={f1:.3f}(Phi0), df01/dt={df01df*dfdt:.1f} (MHz/us), \n df/dt={dfdt:.3f} (Phi0/us), Gamma1={gamma1:.3f}, Gamma2={gamma2:.3f}', fontsize=16)

eps_list = [eps_t(t, exp_args) for t in tlist]
fig, ax1 = plt.subplots(figsize=(12,8))
ax1.plot(tlist, eps_list)
ax1.set_xlabel('Time (us)', fontsize=20)
ax1.set_ylabel('eps (MHz)', fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax1.legend(("R state", "L state"), loc=0, fontsize=20);
ax1.grid()

plot_eps_vs_time(delta, eps_list, tlist)

# Out and back

## function

In [9]:
# def eps_t_outandback_linear(t, p):
#     """ Ideal linear ramp, idle and ramp down"""
#     t1 = (p.f2 - p.f1) / p.dfdt
#     if t <= t1:
#         f = t * p.dfdt + p.f1
#     elif t<= t1 + p.idle:
#         f = p.f2
#     else:
#         f = -(t-(t1 + p.idle)) * p.dfdt + p.f2
        
#     eps = p.df01df * f
    
#     return eps

# def eps_t_exponential(t, args):
#     """ Not very good. Exponential ramp up and linear ramp-down"""
#     factor=0.5

#     if t <= args['t1'] + args['idle']:        
#         s = (1-np.exp(-t/factor))*(args['f2']-args['f1']) + args['f1']
        
#     else:
#         t2 = args['t1'] + args['idle']
#         s = -args['dfdt']*(t-t2) + (1-np.exp(-t2/factor))*(args['f2']-args['f1']) + args['f1']
                        
       
#     return s *args['df01df'] 


def eps_list_no_filter(tlist, eps_list, filter_args):
    return eps_list

def eps_list_RLC(tlist, eps_list, params):
    """
    Create nonideal eps_t.
    Args: 
        tlist: time list
        eps_list: ideal eps list
    Return: nonideal_eps_list
    """

    t = tlist
    u = eps_list - eps_list[0]
    L, C, factor = (params.filter_params['L']
                    , params.filter_params['C']
                    , params.filter_params['factor'])
    
    RLC = np.array([1, L*factor, C*factor])   # R,L,C
    num, den = [0,0,1], [RLC[1]*RLC[2], RLC[0]*RLC[2],1]
    # num, den = [0,RLC[0]*RLC[2],0], [RLC[1]*RLC[2], RLC[0]*RLC[2],1]

    # z, p, k = scipy.signal.besselap(1)
    # z, p, k = scipy.signal.buttap(1)
    # num, den = scipy.signal.zpk2tf(z, p, k)

    system = scipy.signal.lti(num, den)
    tout, y, x = scipy.signal.lsim(system, u, t)

    return np.array(y + eps_list[0], dtype=float)

# def eps_t_with_extension(t, args):
#     if t <= args['t1']:
#         f = t * args['dfdt'] + args['f1']        
#     elif t<= args['t1'] + args['idle']:
#         f = args['f2']
#     elif t<=args['t1']*2 + args['idle']:
#         f = -(t-(args['t1']+args['idle'])) * args['dfdt'] + args['f2']
#     else:
#         f = args['f1']
        
#     eps = args['df01df'] * f
    
#     return eps

# def get_tlist_outandback(p):
#     """ p: ExpParams instance"""
#     t1 = (p.f2 - p.f1) / p.dfdt
#     idle = p.idle
#     n1, n2, n3 = np.round(t1/p.dt), np.round(p.idle/p.dt), np.round(t1/p.dt)
    
#     tlist1 = np.linspace(0, t1, n1+1)
#     idle_list_dt = np.linspace(0, idle, n2+1)
#     tlist2 = np.linspace(0, t1, n3+1)
#     if p.extension != 0:
#         n4 = np.round(p.extension/p.dt)
#         tlist3 = np.linspace(0, p.extension, n4+1)
#         tlist = np.concatenate((tlist1, idle_list_dt[1:]+t1, tlist2[1:]+t1+idle, tlist3[1:]+2*t1+idle))
#     else:
#         tlist = np.concatenate((tlist1, idle_list_dt[1:]+t1, tlist2[1:]+t1+idle))
    
#     return tlist

def get_tlist_eps_list_outandback_linear(params):
    """ Ideal linear ramp, idle and ramp down"""

    t1 = (params.f2 - params.f1) / params.dfdt
    idle = params.idle
    dt = params.dt
    
    # get tlist
    n1, n2, n3 = np.round(t1/dt), np.round(idle/dt), np.round(t1/dt)
    
    tlist1 = np.linspace(0, t1, n1+1)
    tlist2 = np.linspace(0, idle, n2+1)
    tlist3 = np.linspace(0, t1, n3+1)
    if params.extension != 0:
        n4 = np.round(params.extension/dt)
        tlist4 = np.linspace(0, params.extension, n4+1)
        tlist = np.concatenate((tlist1, tlist2[1:]+t1, tlist3[1:]+t1+idle, tlist4[1:]+2*t1+idle))
    else:
        tlist = np.concatenate((tlist1, tlist2[1:]+t1, tlist3[1:]+t1+idle))
    
    # get eps_list
    f1 = tlist1 * params.dfdt + params.f1
    f2 = params.f2 * np.ones(tlist2.size-1)
    f3 = -tlist3[1:] * params.dfdt + params.f2
    if params.extension !=0:
        f4 = params.f1 * np.ones(tlist4.size-1)
        f_list = np.concatenate((f1, f2, f3, f4))
    else:
        f_list = np.concatenate((f1, f2, f3))
    eps_list = params.df01df * f_list
    
    return (tlist, eps_list)

def out_idle_back(params, eps_filter):
    """ Calculate R state probability in out-idle-back measurement
        Args:
            params: ExpParams instance
            eps_list_filter: a filter function that converts ideal eps_list to 
            new eps_list
        Return:
            numpy array with float datatype.
    """
    # compute
    p_ex_idle_list = np.ones(params.idle_list.size)
    for j, idle in enumerate(params.idle_list):
        params.idle = idle
        tlist, eps_list = get_tlist_eps_list_outandback_linear(params)        
        eps_list_filter = eps_filter(tlist, eps_list, params) # after conversion
        p_ex = qubit_evolve(tlist, eps_list_filter, params)
        p_ex_idle_list[j] = p_ex[-1]
        
    return p_ex_idle_list, tlist, eps_list, eps_list_filter


In [ ]:
# def qubit_evolve_nonideal(delta, gamma1, gamma2, psi0, eps_t, tlist, H_args, n_th=0):
    """
    eps_t : Function that represents eps(t) in -eps(t)/2*sz, 
            i.e., defines how energy difference changes
    
    Return: expectation values
    """
    # Hamiltonian
    sz = sigmaz()
    sm = destroy(2)

    H0 = - delta/2.0 * sigmax() # - eps0/2.0 * sigmaz()     

    # collapse operators
    c_op_list = []
    n_th = 0.0 # zero temperature

    # relaxation
    rate1 = gamma1 * (1 + n_th)
    # excitation
    rate2 = gamma1 * n_th
    # dephasing 
    rate3 = gamma2
    
    c_op_list = [sqrt(rate1) * sm, sqrt(rate2) * sm.dag(), sqrt(rate3) * sz]

    eps_list = [eps_t(t, H_args) for t in tlist]
    
    nonideal_eps_list = eps_t_RLC(tlist, eps_list, H_args['L'], H_args['C'], H_args['factor'])
    
    H = [H0, [-1/2.0 * sz, nonideal_eps_list]]
    output = mesolve(H, psi0, tlist, c_op_list, [sm.dag() * sm], args=H_args)  


    return output.expect[0]

## a fixed idle

In [173]:
# pulse out and back with with a fixed idle, 1D

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
params.dfdt = 0.01/1 # in Phi0/us
params.f1 = -0.002
params.f2 = 0.0000
params.dt = 0.002    # us
params.gamma1 = 0  # relaxation rate
params.gamma2 = 0  # dephasing  rate
params.n_th = 0 
params.extension = 0 # us, extra time after ramp-down.
params.idle = 2
params.psi0 = basis(2,0)      # initial state

start_time = time.time()
tlist, eps_list = get_tlist_eps_list_outandback_linear(params)
p_ex = qubit_evolve(tlist, eps_list, params)
print('time elapsed = ' + str(time.time() - start_time))

# plot
fig, (ax,ax1) = plt.subplots(2,1,figsize=(12,10))
ax.plot(tlist, np.real(p_ex), 'b', tlist, np.real(1-p_ex), 'r')
ax.set_xlabel('Time (us)', fontsize=20)
ax.set_ylabel('Occupation probability', fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Pulse out and back, idle={params.idle:.3f} (us) \n'
             + f', delta={params.delta} (MHz), f1={params.f1*1e3:.3f}(mPhi0),  f2={params.f2*1e3:.3f}f(mPhi0), df01/dt={params.df01df*params.dfdt:.3f} (MHz/us)\n'
             + f', df/dt={params.dfdt:.3f} (Phi0/us), Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f}', fontsize=16)

ax.legend(("R state", "L state"), loc=0, fontsize=20);
ax.grid()

ax1.plot(tlist, eps_list)
ax1.set_xlabel('Time (us)', fontsize=20)
ax1.set_ylabel('eps (MHz)', fontsize=20)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.legend(("R state", "L state"), loc=0, fontsize=20);
ax1.grid()

# plot_eps_vs_time(delta, eps_list, tlist)

time elapsed = 0.08423066139221191


## vs idle, 1D

In [11]:
# out and back
# vs idle , 1D

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
params.dfdt = 0.04/0.1 # in Phi0/us
params.f1 = -0.001
params.f2 = 0.0000
params.dt = 0.002    # us
params.gamma1 = 1  # relaxation rate
params.gamma2 = 20  # dephasing  rate
params.n_th = 0 
params.extension = 0 # us, extra time after ramp-down.
params.idle_list = np.linspace(0, 1, 51)
params.psi0 = basis(2,0)      # initial state

filter_params = {'L':1, 'C':10 , 'factor':0.01} # L, C, factor = 1, 10, 0.005
params.filter_params = filter_params

# choose eps_t
# TODO add 3: eps_list_exponential}
eps_filters= {1:eps_list_no_filter, 2:eps_list_RLC}
eps_filter = eps_filters[1]

# compute
p_ex_idle_list, tlist, eps_list, eps_list_filter = out_idle_back(params, eps_filter)

# plot
fig, (ax,ax1) = plt.subplots(2,1,figsize=(12,10))
ax.plot(params.idle_list, p_ex_idle_list, 'b')
ax.set_xlabel('Idle time (us)',fontsize=20)
ax.set_ylabel('Ocupation Probability',fontsize=20)
ax.set_ylim((-0.05,1.05))
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.legend(("R state",), loc=0, fontsize=20);
ax.set_title(f'Pulse out and back \n'
             + f', delta={params.delta} (MHz), f1={params.f1*1e3:.3f}(mPhi0),  f2={params.f2*1e3:.3f}f(mPhi0), df01/dt={params.df01df*params.dfdt:.3f} (MHz/us)\n'
             + f', df/dt={params.dfdt:.3f} (Phi0/us), Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f}', fontsize=16)

ax1.plot(tlist, eps_list)
if eps_filter != eps_filters[1]:
    ax1.plot(tlist, eps_list_filter)
ax1.set_xlabel('Time (us)', fontsize=20)
ax1.set_ylabel('eps (MHz)', fontsize=20)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.grid()

plot_eps_vs_time(tlist, eps_list_filter, params)

tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?
tlist not uniform?


In [146]:
eps_list_test = [eps_t_outandback_linear(t, params) for t in tlist]
plt.plot(tlist, eps_list_test,'bo-')

tlist_t, eps_list_t = get_tlist_eps_list_outandback_linear(params)
plt.plot(tlist_t, eps_list_t, 'r*-')



## f2 vs idle, 2D

In [16]:
# out and back
# f2 vs idle , 2D

import nmmn.plots
from IPython.display import clear_output

parula=nmmn.plots.parulacmap() 
turbo=nmmn.plots.turbocmap() 

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
params.dfdt = 0.001/0.1 # in Phi0/us
params.f1 = -0.001
params.dt = 0.002    # us
params.gamma1 = 1  # relaxation rate
params.gamma2 = 20  # dephasing  rate
params.n_th = 0 
params.extension = 0 # us, extra time after ramp-down.
params.idle_list = np.linspace(0, 1, 51)
params.f2_list = np.linspace(-0.001, 0.003, 81) 
params.psi0 = basis(2,0)      # initial state

filter_params = {'L':1, 'C':10 , 'factor':0.01} # L, C, factor = 1, 10, 0.005
params.filter_params = filter_params

# choose eps_t
# TODO add 3: eps_list_outbandback_exponential}
eps_filters= {1:eps_list_no_filter, 2:eps_list_RLC}
eps_filter = eps_filters[2]

# compute
data = np.ones((params.f2_list.size, params.idle_list.size))
for i, f2 in enumerate(params.f2_list):
    clear_output()
    print(f'flux={i+1}/{params.f2_list.size}')
   
    params.f2 = f2
    p_ex_idle_list, tlist, eps_list, eps_list_filter = out_idle_back(params, eps_filter)    
    data[i,:] = p_ex_idle_list

# plot
idle_mat, f2_mat = np.meshgrid(params.idle_list, params.f2_list)
fig, ax = plt.subplots(figsize=(12,8))
color = ax.pcolormesh(idle_mat, f2_mat*1e3, data, cmap=parula)
ax.set_xlabel('Idle time (us)',fontsize=20)
ax.set_ylabel(r'$\Delta$Flux (m$\Phi_0$)',fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Pulse out and back, delta={delta} (MHz), f1={f1*1e3:.3f}(mPhi0),\n df01/df={df01df:.3f} (MHz/Phi), df01/dt={df01df*dfdt:.3f} (MHz/us), df/dt={dfdt:.3f} (Phi0/us),\n Gamma1={gamma1:.3f}, Gamma2={gamma2:.3f}', fontsize=16)

cb = fig.colorbar(color)
cb.set_label('|R> occupation', fontsize=20)
cb.ax.tick_params(labelsize=18)

plot_eps_vs_time(tlist, eps_list_filter, params)

flux=1/81


TypeError: len() of unsized object

In [ ]:
from numpy import min, max
from scipy import linspace
from scipy.signal import lti, step, impulse

# making transfer function
# example from Ogata Modern Control Engineering 
# 4th edition, International Edition page 307

# num and den, can be list or numpy array type
num = [6.3223, 18, 12.811] 
den = [1, 6, 11.3223, 18, 12.811]

tf = lti(num, den)

# get t = time, s = unit-step response
t, s = step(tf)

# recalculate t and s to get smooth plot
t, s = step(tf, T = linspace(min(t), t[-1], 500))

# get i = impulse
t, i = impulse(tf, T = linspace(min(t), t[-1], 500))

from matplotlib import pyplot as plt

plt.plot(t, s, t, i)
plt.title('Transient-Response Analysis')
plt.xlabel('Time(sec)')
plt.ylabel('Amplitude')
plt.hlines(1, min(t), max(t), colors='r')
plt.hlines(0, min(t), max(t))
plt.xlim(xmax=max(t))
plt.legend(('Unit-Step Response', 'Unit-Impulse Response'), loc=0)
plt.grid()


In [ ]:
from scipy.signal import butter, lfilter, freqz
import scipy
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

t = np.linspace(0,1,100)
sig = np.tanh(np.linspace(0,2,100))
# sig = np.exp(-t**2)
# sig = np.tanh(t/0.4)
cutoff = 0.01
fs = 0.1
order =5
# proc = butter_lowpass_filter(sig, cutoff, fs, order)
# f = scipy.fft(sig)
# f[25:275] = 0
# ff = scipy.ifft(f)
# proc = sig * np.exp(-((t-1.5)/2)**2)
plt.plot(t, sig)
# plt.plot(t,sig, t,np.absolute(ff))

In [ ]:
L, C = 2, 4
factor = 0.01
nonideal =  nonideal_eps_t(tlist, eps_list, L, C, factor )

# plot
fig, (ax1) = plt.subplots(1, 1, figsize=(12,8))

ax1.plot(tlist, eps_list, tlist, nonideal)
ax1.set_xlabel('t',fontsize=20)
ax1.set_ylabel(r'Response',fontsize=20)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)

## out and back with exponential

In [3]:
def eps_t_exponential(idle, params):
    """ Not very good. Exponential ramp up and linear ramp-down"""
    
    idle_list = np.arange(0, idle+params.dt, params.dt)
    f_list = ((1 - params.exp_A * np.exp(-idle_list/params.exp_tau))-(1-params.exp_A) )*(params.f2 - params.f1)/params.exp_A + params.f1
    
    f0 = f_list[-1] 
    tlist_trailing = np.arange(params.dt, params.exp_tau *3 , params.dt)
    f_trailing = np.exp(-tlist_trailing/params.exp_tau) * (f0 - params.f1) + params.f1
    
    f_list = np.concatenate( (f_list, f_trailing))
                        
    eps_list = f_list  * params.df01df
    tlist = np.concatenate((idle_list, tlist_trailing + idle))
    
    return tlist, eps_list
def eps_t_exponential_double(idle, params):
    """ double exponential. fast rise + slow rise for ramp up. fast ramp-down"""
    idle_list = np.arange(0, idle+params.dt, params.dt)
    f_list = ((1 - (params.exp_A * np.exp(-idle_list/params.exp_tau)
                    + (1-params.exp_A) * np.exp(-idle_list/params.exp_tau1))))*(params.f2 - params.f1) + params.f1
    
    f0 = f_list[-1] 
    tlist_trailing = np.arange(params.dt, params.exp_tau *3 , params.dt)
    f_trailing = np.exp(-tlist_trailing/params.exp_tau) * (f0 - params.f1) + params.f1
    
    f_list = np.concatenate( (f_list, f_trailing))
                        
    eps_list = f_list  * params.df01df
    tlist = np.concatenate((idle_list, tlist_trailing + idle))
    
    return tlist, eps_list

In [4]:
# out and back
# vs idle , 1D

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
# params.dfdt = 0.001/0.3 # in Phi0/us
params.f1 = -0.002
params.f2 = 0.001
params.dt = 0.002    # us
params.gamma1 = 0.3  # relaxation rate
params.gamma2 = 10  #dephasing  rate
params.n_th = 0 
params.extension = 0 # us, extra time after ramp-down.
params.idle_list = np.linspace(0, 4, 51)
params.psi0 = basis(2,0)      # initial state
# params.exp_tau = 0.2
# params.exp_A = 1
params.exp_tau = 0.2
params.exp_tau1 = 3
params.exp_A = 0.8  #  0 <=exp_A <=1


# compute
p_ex_idle_list = np.ones(params.idle_list.size)
for j, idle in enumerate(params.idle_list):
    tlist, eps_list = eps_t_exponential_double(idle, params)
    p_ex = qubit_evolve(tlist, eps_list, params)
    p_ex_idle_list[j] = p_ex[-1]

# plot
fig, (ax,ax1) = plt.subplots(2,1,figsize=(12,10))
ax.plot(params.idle_list, p_ex_idle_list, label=f'gamma2={params.gamma2}')
ax.set_xlabel('Idle time (us)',fontsize=20)
ax.set_ylabel('0R ocupation Probability',fontsize=20)
ax.set_ylim((-0.05,1.05))
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.legend(("R state",), loc=0, fontsize=20);
ax.set_title(f'Pulse out and back \n'
             + f', delta={params.delta} (MHz), f1={params.f1*1e3:.3f}(mPhi0),  f2={params.f2*1e3:.3f}f(mPhi0), df01/df={params.df01df:.1f} (MHz/us)\n'
             + f', Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f},exp_tau={params.exp_tau} us', fontsize=16)
ax.legend(fontsize=18, loc=1)

ax1.plot(tlist, eps_list)
ax1.set_xlabel('Time (us)', fontsize=20)
ax1.set_ylabel('eps (MHz)', fontsize=20)
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.grid()

        
# plot_eps_vs_time(tlist, eps_list, params)

In [131]:
ax.legend(fontsize=14, loc=1)

In [5]:
def eps_t_exponential(idle, params):
       
    idle_list = np.arange(0, idle+params.dt, params.dt)
    f_list = ((1 - params.exp_A * np.exp(-idle_list/params.exp_tau))-(1-params.exp_A) )*(params.f2 - params.f1)/params.exp_A + params.f1
    
    f0 = f_list[-1] 
    tlist_trailing = np.arange(params.dt, params.exp_tau *3 , params.dt)
    f_trailing = np.exp(-tlist_trailing/params.exp_tau) * (f0 - params.f1) + params.f1
    
    f_list = np.concatenate( (f_list, f_trailing))
                        
    eps_list = f_list  * params.df01df
    tlist = np.concatenate((idle_list, tlist_trailing + idle))
    
    return tlist, eps_list

def eps_t_exponential_double(idle, params):
    """ double exponential. fast rise + slow rise for ramp up. fast ramp-down"""
    idle_list = np.arange(0, idle+params.dt, params.dt)
    f_list = ((1 - (params.exp_A * np.exp(-idle_list/params.exp_tau)
                    + (1-params.exp_A) * np.exp(-idle_list/params.exp_tau1))))*(params.f2 - params.f1) + params.f1
    
    f0 = f_list[-1] 
    tlist_trailing = np.arange(params.dt, params.exp_tau *3 , params.dt)
    f_trailing = np.exp(-tlist_trailing/params.exp_tau) * (f0 - params.f1) + params.f1
    
    f_list = np.concatenate( (f_list, f_trailing))
                        
    eps_list = f_list  * params.df01df
    tlist = np.concatenate((idle_list, tlist_trailing + idle))
    
    return tlist, eps_list

params.exp_tau = 0.05
params.exp_tau1 = 1
params.exp_A = 1  #  0 <=exp_A <=1

# a, b=eps_t_exponential(2, params)
a, b=eps_t_exponential_double(2, params)
fig, ax = plt.subplots(1,1, figsize=(9,6))
ax.plot(a,b)
ax.set_xlabel('Time (us)',fontsize=20)
ax.set_ylabel(r'eps (MHz)',fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Out-idle-back pulse\n tau ={params.exp_tau} us', fontsize=16)


Text(0.5, 1.0, 'Out-idle-back pulse\n tau =0.05 us')

In [7]:
# out and back
# f2 vs idle , 2D

import nmmn.plots
from IPython.display import clear_output

parula=nmmn.plots.parulacmap() 
turbo=nmmn.plots.turbocmap() 

# parameters
params =ExpParams()
params.delta = 10   #  MHz
params.df01df= 5000/0.015 # define energy vs flux slope MHz/Phi0
# params.dfdt = 0.01/0.1 # in Phi0/us
params.f1 = -0.001
params.dt = 0.002    # us
params.gamma1 = 1  # relaxation rate
params.gamma2 = 10  # dephasing  rate
params.n_th = 0 
params.extension = 0 # us, extra time after ramp-down.
params.idle_list = np.linspace(0, 1, 51)
params.f2_list = np.linspace(-0.001, 0.003, 81) 
# params.f2_list = np.linspace(-0.001, 0.001, 41) 
params.psi0 = basis(2,0)      # initial state
# params.exp_tau = 0.4
# params.exp_A = 1
params.exp_tau = 0.05
params.exp_tau1 = 1
params.exp_A = 1  #  0 <=exp_A <=1


# compute
data = np.ones((params.f2_list.size, params.idle_list.size))
for i, f2 in enumerate(params.f2_list):
    clear_output()
    print(f'flux={i+1}/{params.f2_list.size}')
      
    params.f2 = f2
    print(f2)
    p_ex_idle_list = np.ones(params.idle_list.size)
    for j, idle in enumerate(params.idle_list):
        tlist, eps_list = eps_t_exponential_double(idle, params)
        p_ex = qubit_evolve(tlist, eps_list, params)
        p_ex_idle_list[j] = p_ex[-1]
    
#     p_ex_idle_list, tlist, eps_list, eps_list_filter = out_idle_back(params, eps_filter)    
    data[i,:] = p_ex_idle_list

# plot
idle_mat, f2_mat = np.meshgrid(params.idle_list, params.f2_list)
fig, ax = plt.subplots(figsize=(12,8))
color = ax.pcolormesh(idle_mat, f2_mat*1e3, data, cmap=parula)
ax.set_xlabel('Idle time (us)',fontsize=20)
ax.set_ylabel(r'$\Delta$Flux (m$\Phi_0$)',fontsize=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_title(f'Pulse out and back, delta={params.delta} (MHz), f1={params.f1*1e3:.3f}(mPhi0)\n, exp_tau={params.exp_tau} (us),exp_tau1={params.exp_tau1} (us), exp_A={params.exp_A}\n df01/df={params.df01df:.3f} (MHz/Phi),\n Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f}', fontsize=16)

cb = fig.colorbar(color)
cb.set_label('|R> occupation', fontsize=20)
cb.ax.tick_params(labelsize=18)

plot_eps_vs_time(tlist, eps_list, params)

flux=81/81
0.003


In [82]:
ax.set_title(f'Pulse out and back, delta={params.delta} (MHz), f1={params.f1*1e3:.3f}(mPhi0), exp_tau={params.exp_tau} (us)\n df01/df={params.df01df:.3f} (MHz/Phi), (Phi0/us),\n Gamma1={params.gamma1:.3f}, Gamma2={params.gamma2:.3f}', fontsize=16)

Text(0.5, 1, 'Pulse out and back, delta=10 (MHz), f1=-1.000(mPhi0), exp_tau=0.4 (us)\n df01/df=333333.333 (MHz/Phi), (Phi0/us),\n Gamma1=1.000, Gamma2=10.000')

In [33]:
t = np.linspace(0,1, 101)
tau1 = 0.4
tau2 = 0.01
y = -(0.2*np.exp(-t/tau1) + 1.8*np.exp(-t/tau2))/2
plt.plot(t, y)
plt.grid()
plt.legend()

# old

In [ ]:
np.unravel_index(1,(2,2))

In [ ]:
options = Options()
print(options)

In [ ]:
# Rabi oscillations
H0 = 5000/2 * sigmaz()
H1 = sigmax()

psi0 = basis(2, 0)
tlist = np.linspace(0,0.01, 300)
H = [H0, [H1,'w*cos(5000*t)']]
args = {'w':100}
# H = [H0]
out = mesolve(H, psi0,tlist, [], [sigmax(), sigmay(),sigmaz()], args=args)
exp_x, exp_y, exp_z = out.expect[0], out.expect[1], out.expect[2]

fig, ax = plt.subplots()
ax.plot(tlist, exp_x, tlist, exp_y, tlist, exp_z)
ax.legend(('x', 'y', 'z'))

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([exp_x, exp_y, exp_z], meth='l')
bloch.show()

In [ ]:
# into rotating frame
H_R = lambda t: (-1j*H0*t).expm()

# Rabi oscillations
H0 = 5000/2 * sigmaz()
H1 = sigmax()

psi0 = basis(2, 0)
tlist = np.linspace(0,0.01, 300)
H = [H0, [H1,'w*cos(5000*t)']]
args = {'w':100}
# H = [H0]
out = mesolve(H, psi0,tlist, [], [], args=args)

psi= [H_R(t)*out.states*psi0 for t in tlist]

exp_x, exp_y, exp_z = expect(sigmax(), psi), expect(sigmay(), psi),expect(sigmaz(), psi)

fig, ax = plt.subplots()
ax.plot(tlist, exp_x, tlist, exp_y, tlist, exp_z)
ax.legend(('x', 'y', 'z'))

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([exp_x, exp_y, exp_z], meth='l')
bloch.show()

Hi Ganesh

If you calculate the operator for unitary evolution you will get a 2x2 qobj propagator that you can multiply your initial state (ket) to get the final state:

T = ...
U = propagator(H, T, [], H_args={..})
psi0 = basis(2.0)
psi_T = U * psi0

If you include collapse operators you will get a 4x4 qobj propagator that describes how the density matrix in vector form is transform. So you need to convert your initial state into a density matrix, like this: rho0 = ket2dm(psi0), and then convert it to vector form. We don't have any pretty functions for this yet so its a bit of a trick, which goes something like:

U = propagator(H, T, c_ops, H_args={...})
rho0 = ket2dm(psi0)
rho_vec = mat2vec(rho0.full())
rho_T = Qobj(vec2mat(U.data * rho_vec))

You could also do the same with with Qobj instead of the underlaying sparse matrix data by doing something like:

def rho_vector_form(rho):
    return Qobj(mat2vec(rho.full()), dims=[[rho.dims[0]] * 2, [1]])
def rho_matrix_form(rho_vec):
    return Qobj(vec2mat(rho_vec.full()))

rho_T_vec = U * rho_vector_form(rho0)
rho_T = rho_matrix_form(rho_T_vec)


Hope this helps,
Rob